**Federated Model Training**

In [18]:
smlp_global = SimpleMLP()
global_model = smlp_global.build(784, 10)
loss=[]
acc=[]
#commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)

    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784, 10)
        local_model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=metrics)

        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()

    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
        loss.append(global_loss)
        acc.append(global_acc)

132/132 [==============================] - 0s 2ms/step
comm_round: 0 | global_acc: 87.762% | global_loss: 1.676094651222229
132/132 [==============================] - 0s 2ms/step
comm_round: 1 | global_acc: 90.643% | global_loss: 1.6223770380020142
132/132 [==============================] - 0s 2ms/step
comm_round: 2 | global_acc: 91.667% | global_loss: 1.5997369289398193
132/132 [==============================] - 0s 3ms/step
comm_round: 3 | global_acc: 92.429% | global_loss: 1.5877500772476196
132/132 [==============================] - 1s 4ms/step
comm_round: 4 | global_acc: 92.667% | global_loss: 1.5782935619354248
132/132 [==============================] - 1s 3ms/step
comm_round: 5 | global_acc: 93.333% | global_loss: 1.5716933012008667
132/132 [==============================] - 1s 4ms/step
comm_round: 6 | global_acc: 93.500% | global_loss: 1.5661882162094116
132/132 [==============================] - 0s 2ms/step
comm_round: 7 | global_acc: 93.619% | global_loss: 1.5622512102127075
1

In [19]:
global_model.save("F:/SegmentationModel/UNET/model-sdc-seg-v12.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
